In [1]:
from clickhouse_driver import Client
import pandas as pd


In [2]:
client = Client(host='localhost')
# Test connection
ping = client.execute("SELECT 'Connection successful!'")
print(ping)

[('Connection successful!',)]


<!-- ## Создаем таблицу `employee_data` -->
### Creating the `employee_data` table

In [3]:
q = """
CREATE TABLE IF NOT EXISTS employee_data (
    Age Int32,
    Salary Int32,
    Experience Int32,
    City String,
    Gender String,
    Profession String,
    Industry String,
    Satisfaction_Level Float32,
    Project_Count Int32,
    Bonuses Float32,
    Start_Date Date
) ENGINE = MergeTree()
ORDER BY Age;
"""

client.execute(q)


[]

<!-- ## Заполняем таблицу данными  -->
### Filling the table with data

In [4]:
q = """
INSERT INTO employee_data (
    Age, 
    Salary, 
    Experience, 
    City, 
    Gender, 
    Profession, 
    Industry, 
    Satisfaction_Level, 
    Project_Count, 
    Bonuses, 
    Start_Date
) 
VALUES
(35, 70000, 10, 'Moscow', 'M', 'Engineer', 'Energy', 8.2, 12, 120000, '2020-01-11'),
(45, NULL, 15, 'SPb', 'F', 'Manager', 'Finance', 6.8, NULL, 150000, '2020-02-03'),
(29, 500000, 7, 'Novosibirsk', 'M', 'Developer', 'Technology', 9.0, 22, NULL, '2020-03-21'),
(NULL, 80000, NULL, 'Yekaterinburg', 'F', 'Analyst', 'Technology', 7.5, 14, 130000, '2020-04-05'),
(40, 850000, 14, 'Kazan', 'M', 'Engineer', 'Natural Resources', NULL, 20, 675000, '2020-05-04'),
(33, NULL, 11, 'Omsk', 'F', 'Senior Specialist', 'Natural Resources', 7.0, 17, 125000, NULL),
(28, 60000, NULL, 'Samara', 'M', 'Technician', 'Energy', 8.9, 23, NULL, '2020-07-01'),
(36, 780000, 13, 'Krasnoyarsk', 'F', 'Manager', 'Technology', 6.5, NULL, 130000, '2020-08-01'),
(NULL, 92000, 16, 'Volgograd', 'M', 'Analyst', 'AI', NULL, 15, 140000, '2020-09-06'),
(31, 670000, 9, NULL, 'F', 'Developer', 'Finance', 8.3, 21, 120000, '2019-10-07'),
(35, NULL, NULL, 'Moscow', 'M', 'Engineer', 'Energy', 8.2, NULL, 120000, NULL),
(19, 67787, 9, 'Saratov', 'M', 'Manager', 'Finance', NULL, 21, 120150, '2021-16-01'),
(30, 95000, 14, 'Kazan', 'M', 'Engineer', 'Natural Resources', 8.1, 20, 145000, '2020-05-01'),
(34, 40000, 6, 'Moscow', 'F', 'Analyst', 'Finance', 8.2, 12, 120000, '2020-01-08'),
(15, NULL, 15, 'SPb', 'F', 'Manager', 'Finance', 6.8, NULL, 150000, '2018-02-01'),
(79, 520000, 7, 'Novosibirsk', 'F', 'Developer', 'Finance', 9.0, 22, NULL, '2016-03-21'),
(NULL,60000, NULL, 'Yekaterinburg', 'F', 'Analyst', 'AI', 7.5, 14, 130000, '2020-04-23'),
(20, 351000, 14, 'Kazan', 'F', 'Engineer', 'Natural Resources', NULL, 20, 675000, '2019-05-06'),
(53, NULL, 11, 'Omsk', 'F', 'Senior Specialist', 'Natural Resources', 7.0, 17, 125000, NULL),
(48, 20000, NULL, 'Samara', 'M', 'Technician', 'AI', 8.9, 23, NULL, '2021-07-08'),
(36, 80000, 13, 'Krasnoyarsk', 'M', 'Manager', 'Technology', 6.5, NULL, 130000, '2019-08-11'),
(NULL, 12000, 16, 'Volgograd', 'M', 'Analyst', 'AI', NULL, 15, 140000, '2017-09-19'),
(21, 67000, 9, NULL, 'F', 'Developer', 'Finance', 8.3, 21, 120000, '2021-10-16'),
(65, NULL, NULL, 'Moscow', 'M', 'Engineer', 'Energy', 8.2, NULL, 120000, NULL),
(39, 60700, 9, 'Saratov', 'M', 'Manager', 'Finance', NULL, 21, 120150, '2022-10-13'),
(36, 90000, 14, 'Kazan', 'M', 'Analyst', 'AI', 8.1, 20, 145000, '2010-05-11');

"""

client.execute(q)

[]

#### Function to represent the data in the table

In [5]:
def transform_to_dataframe(data, columns):
    return pd.DataFrame(data, columns=columns)


In [6]:
q = """
select * 
from employee_data

"""

result = client.execute(q)

columns = client.execute("DESCRIBE TABLE employee_data")
columns = [desc[0] for desc in columns]
df = transform_to_dataframe(result, columns)
df.head()

,Age,Salary,Experience,City,Gender,Profession,Industry,Satisfaction_Level,Project_Count,Bonuses,Start_Date
0,0,80000,0,Yekaterinburg,F,Analyst,Technology,7.5,14,130000.0,2020-04-05
1,0,92000,16,Volgograd,M,Analyst,AI,0.0,15,140000.0,2020-09-06
2,0,60000,0,Yekaterinburg,F,Analyst,AI,7.5,14,130000.0,2020-04-23
3,0,12000,16,Volgograd,M,Analyst,AI,0.0,15,140000.0,2017-09-19
4,15,0,15,SPb,F,Manager,Finance,6.8,0,150000.0,2018-02-01


### Working with time data

<!-- Извлечение года, месяца и дня из даты -->
#### Extracting the year, month, and day from the date


In [7]:
q = """
SELECT 
    Start_Date,
    toYear(Start_Date) AS Year,
    toMonth(Start_Date) AS Month,
    toDayOfMonth(Start_Date) AS Day
FROM employee_data;
"""

result = client.execute(q)

columns = ['Start_Date', 'Year', 'Month', 'Day']
df = transform_to_dataframe(result, columns)
df.head()

,Start_Date,Year,Month,Day
0,2020-04-05,2020,4,5
1,2020-09-06,2020,9,6
2,2020-04-23,2020,4,23
3,2017-09-19,2017,9,19
4,2018-02-01,2018,2,1


#### Formatting dates into a specific string format, for example, `YYYY_MM_DD`, use the `formatDateTime` function

In [8]:
q = """
SELECT 
    Start_Date,
    formatDateTime(Start_Date, '%Y_%m_%d') AS Formatted_Date
FROM 
    employee_data;
"""

result = client.execute(q)

columns = ['Start_Date', 'Formatted_Date']
df = transform_to_dataframe(result, columns)
df.head()

,Start_Date,Formatted_Date
0,2020-04-05,2020_04_05
1,2020-09-06,2020_09_06
2,2020-04-23,2020_04_23
3,2017-09-19,2017_09_19
4,2018-02-01,2018_02_01


#### Calculate the difference in days between `Start_Date` and the current date using the `dateDiff` function:

In [9]:
q = """
SELECT 
    Start_Date, 
    dateDiff('day', Start_Date, now()) AS days_difference 
FROM employee_data;

"""

result = client.execute(q)

columns = ['Start_Date', 'days_difference']
df = transform_to_dataframe(result, columns)
df.head()

,Start_Date,days_difference
0,2020-04-05,1712
1,2020-09-06,1558
2,2020-04-23,1694
3,2017-09-19,2641
4,2018-02-01,2506


<!-- Получить начало месяца и года для каждой даты, используйте функции `toStartOfMonth()` и `toStartOfYear()` -->
#### Get the start of the month and year for each date using the `toStartOfMonth()` and `toStartOfYear()` functions

In [10]:
q = """
SELECT 
    Start_Date,
    toStartOfMonth(Start_Date) AS Start_of_Month,
    toStartOfYear(Start_Date) AS Start_of_Year
FROM employee_data;
"""

result = client.execute(q)

columns = ['Start_Date', 'Start_of_Month', 'Start_of_Year']
df = transform_to_dataframe(result, columns)
df.head()

,Start_Date,Start_of_Month,Start_of_Year
0,2020-04-05,2020-04-01,2020-01-01
1,2020-09-06,2020-09-01,2020-01-01
2,2020-04-23,2020-04-01,2020-01-01
3,2017-09-19,2017-09-01,2017-01-01
4,2018-02-01,2018-02-01,2018-01-01


<!-- ## Задание (комбинаторные функции)  -->
### Combinatorial functions

<!-- Определить следующее:
1. Среднюю зарплату (`Salary`) для сотрудников с опытом работы больше 10 лет в каждой отрасли.
2. Количество сотрудников в каждой отрасли с уровнем удовлетворённости ниже 6.0.
3. Средний бонус (`Bonuses`) для сотрудников в каждой отрасли, где уровень удовлетворённости больше среднего уровня удовлетворённости по всем отраслям.
4. Найти максимальный и минимальный возраст сотрудников для каждой профессии в каждой отрасли, учитывая только сотрудников, у которых опыт больше 5 лет и зарплата не менее 50000. -->

1. The average salary (`Salary`) for employees with more than 10 years of experience in each industry.
2. The number of employees in each industry with a satisfaction level below 6.0.
3. The average bonus (`Bonuses`) for employees in each industry where the satisfaction level is above the average satisfaction level across all industries.
4. Find the maximum and minimum age of employees for each profession in each industry, considering only employees with more than 5 years of experience and a salary of at least 50000.


In [11]:
q = """
SELECT 
    Industry,
    ROUND(avgIf(Salary, Experience > 10), 2) AS Average_Salary,
    countIf(Satisfaction_Level < 6.0) AS Unsatisfied_Employees,
    ROUND(avgIf(Bonuses, Satisfaction_Level > (
        SELECT avg(Satisfaction_Level)
        FROM employee_data
    )
    ),2) AS Average_Bonuses,
    maxIf(Age, Experience > 5 AND Salary >= 50000) AS Max_Age,
    minIf(Age, Experience > 5 AND Salary >= 50000) AS Min_Age
FROM employee_data
GROUP BY Industry;
"""

result = client.execute(q)

columns = ['Industry', 'Average_Salary', 'Unsatisfied_Employees', 'Average_Bonuses', 'Max_Age', 'Min_Age']
df = transform_to_dataframe(result, columns)
df.head()

,Industry,Average_Salary,Unsatisfied_Employees,Average_Bonuses,Max_Age,Min_Age
0,Finance,0.00,2,110000.00,79,19
1,AI,64666.67,2,91666.67,36,0
2,Energy,NaN,0,90000.00,35,35
3,Technology,430000.00,0,97500.00,36,29
4,Natural Resources,259200.00,2,131666.67,40,20


### Arrays

<!-- 1. Для каждой отрасли найдите список уникальных профессий, которыми заняты сотрудники этой отрасли, без учёта пола (подсказка: `groupUniqArray`)
2. Составьте список уникальных профессий, которыми заняты только женщины в каждой отрасли (подсказка: `groupUniqArrayIf`)
3. Определите количество уникальных профессий среди женщин в каждой отрасли, то есть посчитайте уникальные профессии из списка профессий, где работают женщины (п2) (подсказка: `groupUniq`) -->

1. For each industry, find a list of unique professions that employees of this industry are engaged in, without regard to gender (`groupUniqArray`)
2. Make a list of unique professions that are occupied only by women in each industry (`groupUniqArrayIf`)
3. Determine the number of unique professions among women in each industry, that is, count the unique professions from the list of professions where women work (p2) (`groupUniq`)


In [12]:
q = """
WITH temp AS (
    SELECT 
        Industry,
        groupUniqArray(Profession) AS Unique_F_Professions,
        groupUniqArrayIf(Profession, Gender = 'F') AS Unique_Professions
    FROM employee_data
    GROUP BY Industry
)
SELECT 
    Industry,
    Unique_F_Professions,
    Unique_Professions,
    length(Unique_Professions) AS Unique
FROM temp;
"""

result = client.execute(q)

columns = ['Industry', 'Unique_F_Professions', 'Unique_Professions', 'Unique']
df = transform_to_dataframe(result, columns)
df.head()


,Industry,Unique_F_Professions,Unique_Professions,Unique
0,Finance,"[Manager, Analyst, Developer]","[Manager, Analyst, Developer]",3
1,AI,"[Technician, Analyst]",[Analyst],1
2,Energy,"[Technician, Engineer]",[],0
3,Technology,"[Manager, Analyst, Developer]","[Manager, Analyst]",2
4,Natural Resources,"[Engineer, Senior Specialist]","[Engineer, Senior Specialist]",2


<!-- 1. Соберите массив зарплат для каждой отрасли, включив в него только зарплаты выше 100,000. Отфильтрованный массив отсортируйте в порядке возрастания, чтобы зарплаты располагались от меньших к большим (подсказка: `groupArray` -> `arrayFilter` -> `arraySort`)
2. Создайте альтернативный массив зарплат для каждой отрасли, также включающий только значения выше 100,000, но используя фильтрацию в самой функции `groupArrayIf`. Отсортируйте массив по возрастанию. (подсказка: `groupArrayIf` -> `arraySort`) -->

1. Collect an array of salaries for each industry, including only salaries above 100,000. Sort the filtered array in ascending order so that the salaries are arranged from smallest to largest (`groupArray` -> `arrayFilter` -> `arraySort`)
2. Create an alternative array of salaries for each industry, also including only values above 100,000, but using filtering in the `groupArrayIf` function itself. Sort the array in ascending order. (`groupArrayIf` -> `arraySort`)

In [13]:
q = """
SELECT 
    Industry,
    arraySort(arrayFilter(x -> x > 100000, groupArray(Salary))) AS Filtered_Salaries,
    arraySort(groupArrayIf(Salary, Salary > 100000)) AS Filtered_Salaries_If
FROM employee_data
GROUP BY Industry; 
"""

result = client.execute(q)

columns = ['Industry', 'Filtered_Salaries', 'Filtered_Salaries_If']
df = transform_to_dataframe(result, columns)
df.head()


,Industry,Filtered_Salaries,Filtered_Salaries_If
0,Finance,"[520000, 670000]","[520000, 670000]"
1,AI,[],[]
2,Energy,[],[]
3,Technology,"[500000, 780000]","[500000, 780000]"
4,Natural Resources,"[351000, 850000]","[351000, 850000]"


<!-- 1. Соберите массив зарплат сотрудников в каждой отрасли, отсортировав его по убыванию (подсказка: `arraySort`)
2. Извлеките топ-3 зарплаты для каждой отрасли. Используйте `arraySlice`, чтобы выбрать только первые три значения в отсортированном массиве (подсказка: `arraySlice`) -->

1. Collect an array of employee salaries in each industry, sorting it in descending order (`arraySort`)
2. Extract the top 3 salaries for each industry. Use `arraySlice` to select only the first three values in the sorted array (`arraySlice`)



In [14]:
q = """
SELECT 
    arrayReverseSort(groupArray(Salary)) as Salary_Array,
    arraySlice(arrayReverseSort(groupArray(Salary)), 1, 3) as Top_3_Salaries
FROM employee_data
GROUP BY Industry;
"""

result = client.execute(q)

columns = ['Salary_Array', 'Top_3_Salaries']
df = transform_to_dataframe(result, columns)
df.head()

,Salary_Array,Top_3_Salaries
0,"[670000, 520000, 67787, 67000, 60700, 40000, 0...","[670000, 520000, 67787]"
1,"[92000, 90000, 60000, 20000, 12000]","[92000, 90000, 60000]"
2,"[70000, 60000, 0, 0]","[70000, 60000, 0]"
3,"[780000, 500000, 80000, 80000]","[780000, 500000, 80000]"
4,"[850000, 351000, 95000, 0, 0]","[850000, 351000, 95000]"


### lambda functions

<!-- 1. Составьте массив зарплат для каждой отрасли, включив только те зарплаты, которые превышают 50,000. Для анализа возможного повышения зарплат увеличьте каждое значение на 10% и округлите результат до одного знака после запятой.
2. Составьте массив опыта сотрудников для каждой отрасли, включив только сотрудников с опытом работы более 5 лет. В результате для каждого сотрудника добавьте к их текущему опыту 2 года -->

1. Compile an array of salaries for each industry, including only those salaries that exceed 50,000. To analyze the possibility of a salary increase, increase each value by 10% and round the result to one decimal place.
2. Compile an array of employee experience for each industry, including only employees with more than 5 years of experience. As a result, add 2 years to their current experience

In [15]:
q = """
SELECT 
    Industry,
    arrayMap(x -> (ROUND(x * 1.1 ,1)), arrayFilter(x -> x > 50000, groupArray(Salary))),
    arrayMap(x -> x + 2, arrayFilter(x -> x > 5, groupArray(Experience)))
FROM employee_data
GROUP BY Industry;
"""

result = client.execute(q)

columns = ['Industry', 'Salary', 'Experience']
df = transform_to_dataframe(result, columns)
df.head()

,Industry,Salary,Experience
0,Finance,"[74565.7, 73700.0, 737000.0, 66770.0, 572000.0]","[17, 11, 11, 11, 8, 11, 17, 9]"
1,AI,"[101200.0, 66000.0, 99000.0]","[18, 18, 16]"
2,Energy,"[66000.0, 77000.0]",[12]
3,Technology,"[88000.0, 550000.0, 858000.0, 88000.0]","[9, 15, 15]"
4,Natural Resources,"[386100.0, 104500.0, 935000.0]","[16, 16, 13, 16, 13]"


### CTE - common table expressions

<!-- 1. **CTE**:
- Сгруппируйте данные по `City` и `Gender`.
- Рассчитайте среднюю зарплату (`AVG(Salary)`) и общее количество сотрудников (`COUNT(*)`) для каждой комбинации города и пола.

2. **Основной запрос**:
- Используйте подзапрос, чтобы объединить его результаты с таблицей `employee_data`, сопоставив сотрудников с их городом и полом.
- Отфильтруйте данные так, чтобы вывести только тех сотрудников, у которых зарплата выше средней зарплаты по их городу и полу. -->

1. **CTE**:
- Group the data by `City` and `Gender`.
- Calculate the average salary (`AVG(Salary)`) and the total number of employees (`COUNT(*)`) for each combination of city and gender.

2. **Main query**:
- Use a subquery to combine its results with the `employee_data` table, matching employees with their city and gender.
- Filter the data to display only those employees whose salary is above the average salary for their city and gender.


In [16]:
q = """
with subquery as (
    SELECT
    City, Gender,
    AVG(Salary) as Avg_Salary, 
    COUNT(*) as Employee_Count
    FROM employee_data
    GROUP BY City, Gender
)
SELECT e.*
FROM employee_data as e
JOIN subquery ON subquery.City = e.City and subquery.Gender = e.Gender
WHERE Salary > Avg_Salary;
"""

result = client.execute(q)

columns = client.execute("DESCRIBE TABLE employee_data")
columns = [desc[0] for desc in columns]
df = transform_to_dataframe(result, columns)
df.head()

,Age,Salary,Experience,City,Gender,Profession,Industry,Satisfaction_Level,Project_Count,Bonuses,Start_Date
0,0,80000,0,Yekaterinburg,F,Analyst,Technology,7.5,14,130000.0,2020-04-05
1,0,92000,16,Volgograd,M,Analyst,AI,0.0,15,140000.0,2020-09-06
2,19,67787,9,Saratov,M,Manager,Finance,0.0,21,120150.0,1970-01-01
3,28,60000,0,Samara,M,Technician,Energy,8.9,23,0.0,2020-07-01
4,31,670000,9,,F,Developer,Finance,8.3,21,120000.0,2019-10-07


### Window functions

<!-- 1. Для каждой профессии пронумеруйте сотрудников, начиная с самого высокого уровня зарплаты, используя функцию `ROW_NUMBER()`.
2. Используя функцию `RANK()`, создайте ранжирование сотрудников в каждой отрасли на основе их зарплаты (начиная с самой высокой).
3. Рассчитайте скользящее среднее уровня удовлетворённости для текущей строки и двух предыдущих сотрудников с такой же профессией, упорядоченных по зарплате
4. Рассчитайте cумму зарплат всех сотрудников в рамках города

Условия:

- Отобразите все столбцы из таблицы `employee_data` вместе с добавленными оконными функциями.
- Отсортируйте итоговую таблицу по профессии и уровню зарплаты (в порядке убывания). -->

1. For each profession, number the employees starting from the highest salary level using the `ROW_NUMBER()` function.
2. Using the `RANK()` function, create a ranking of employees in each industry based on their salary (starting with the highest).
3. Calculate the moving average of the satisfaction level for the current line and two previous employees with the same profession, ordered by salary
4. Calculate the amount of salaries of all employees within the city

Conditions:

- Display all columns from the `employee_data` table along with the added window functions.
- Sort the final table by profession and salary level (in desc order).

In [17]:
q = """
SELECT 
    *,
    ROW_NUMBER() OVER (PARTITION BY Profession ORDER BY Salary DESC) AS Row_Number,
    RANK() OVER (PARTITION BY Industry ORDER BY Salary DESC) AS Rank,
    AVG(Satisfaction_Level) OVER (PARTITION BY Profession ORDER BY Salary DESC ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS Sliding_Avg,
    SUM(Salary) OVER (PARTITION BY City) AS City_Salary_Sum
FROM employee_data
ORDER BY Profession, Salary DESC;
"""

result = client.execute(q)

columns = client.execute("DESCRIBE TABLE employee_data")
columns = [desc[0] for desc in columns]
columns.extend(['Row_Number', 'Rank', 'Sliding_Avg', 'City_Salary_Sum'])
df = transform_to_dataframe(result, columns)
df.head()

,Age,Salary,Experience,City,Gender,Profession,Industry,Satisfaction_Level,Project_Count,Bonuses,Start_Date,Row_Number,Rank,Sliding_Avg,City_Salary_Sum
0,0,92000,16,Volgograd,M,Analyst,AI,0.0,15,140000.0,2020-09-06,1,1,0.000000,104000
1,36,90000,14,Kazan,M,Analyst,AI,8.1,20,145000.0,2010-05-11,2,2,4.050000,1386000
2,0,80000,0,Yekaterinburg,F,Analyst,Technology,7.5,14,130000.0,2020-04-05,3,3,5.200000,140000
3,0,60000,0,Yekaterinburg,F,Analyst,AI,7.5,14,130000.0,2020-04-23,4,3,7.700000,140000
4,34,40000,6,Moscow,F,Analyst,Finance,8.2,12,120000.0,2020-01-08,5,6,7.733333,110000


### limit by

<!-- Составьте SQL-запрос, который извлекает уникальные записи сотрудников из таблицы employee_data, ограничивая результаты следующим образом:

1.	Отобразите только одну запись для каждой уникальной комбинации профессии (Profession) и города (City).
2.	Отсортируйте данные так, чтобы для каждой профессии результаты отображались в алфавитном порядке профессий, а в рамках каждой профессии записи городов сортировались по убыванию.
3.	Используйте оператор LIMIT BY для ограничения количества записей до одной на каждую комбинацию профессии и города. -->

Create an SQL query that extracts unique employee records from the employee_data table, limiting the results as follows:

1. Display only one entry for each unique combination of Profession and City.
2. Sort the data so that for each profession the results are displayed in alphabetical order of professions, and within each profession the city records are sorted in descending order.
3. Use the LIMIT BY operator to limit the number of entries to one for each combination of profession and city.


In [18]:
q = """
SELECT *
FROM employee_data
ORDER BY Profession ASC, City DESC
LIMIT 1 BY Profession, City;
"""

result = client.execute(q)

columns = client.execute("DESCRIBE TABLE employee_data")
columns = [desc[0] for desc in columns]
df = transform_to_dataframe(result, columns)
df.head()

,Age,Salary,Experience,City,Gender,Profession,Industry,Satisfaction_Level,Project_Count,Bonuses,Start_Date
0,0,80000,0,Yekaterinburg,F,Analyst,Technology,7.5,14,130000.0,2020-04-05
1,0,92000,16,Volgograd,M,Analyst,AI,0.0,15,140000.0,2020-09-06
2,34,40000,6,Moscow,F,Analyst,Finance,8.2,12,120000.0,2020-01-08
3,36,90000,14,Kazan,M,Analyst,AI,8.1,20,145000.0,2010-05-11
4,29,500000,7,Novosibirsk,M,Developer,Technology,9.0,22,0.0,2020-03-21


### any in grouping

<!-- Сгруппируйте записи по профессии и городу и используйте агрегатную функцию `any` для выбора произвольного значения из каждой группы для следующих столбцов: `Salary`, `Experience`, `Satisfaction_Level` -->

Group the records by profession and city and use the aggregate function `any` to select an arbitrary value from each group for the following columns: `Salary`, `Experience`, `Satisfaction_Level`

In [19]:
q = """
SELECT 
    Profession,
    City,
    any(Salary) AS Salary,
    any(Experience) AS Experience,
    any(Satisfaction_Level) AS Satisfaction_Level
FROM employee_data
GROUP BY Profession, City;
"""

result = client.execute(q)

columns = ['Profession', 'City', 'Salary', 'Experience', 'Satisfaction_Level']
df = transform_to_dataframe(result, columns)
df.head()


,Profession,City,Salary,Experience,Satisfaction_Level
0,Manager,Krasnoyarsk,780000,13,6.5
1,Engineer,Kazan,351000,14,0.0
2,Analyst,Yekaterinburg,80000,0,7.5
3,Manager,SPb,0,15,6.8
4,Developer,,67000,9,8.3


### Strings

<!-- 1. Создайте сокращённое обозначение для каждой записи, используя инициалы города (`City`) и профессии (`Profession`). Инициалы должны быть в формате "C.P.", где первая буква города и первая буква профессии указаны в верхнем регистре, разделённые точками. Например, для города "Moscow" и профессии "Engineer" должно получиться "M.E.".
2. Отфильтруйте записи по названию города. Выберите только те записи, где город начинается с буквы "S" или "K". Используйте регулярное выражение для поиска городов с нужной начальной буквой. -->

1. Create an abbreviated designation for each entry using the initials of the city (`City`) and profession (`Profession`). Initials should be in the format "C.P.", where the first letter of the city and the first letter of the profession are indicated in uppercase, separated by dots. For example, for the city "Moscow" and the profession "Engineer", you should get "M.E.".
2. Filter the entries by the name of the city. Select only those entries where the city starts with the letter "S" or "K". Use a regular expression to search for cities with the desired initial letter.

In [20]:
q = """
SELECT 
    City,
    Profession,
    concat(upper(left(City, 1)), '.', upper(left(Profession, 1)), '.') AS Initials

FROM employee_data
WHERE match(City, '^(S|K)') = 1;
"""

result = client.execute(q)

columns = ['City', 'Profession', 'Initials']
df = transform_to_dataframe(result, columns)
df.head()


,City,Profession,Initials
0,SPb,Manager,S.M.
1,Saratov,Manager,S.M.
2,Kazan,Engineer,K.E.
3,Samara,Technician,S.T.
4,Kazan,Engineer,K.E.


### arrayJoin

<!-- 1. **Внутри CTE**:
   - Сгруппируйте данные по полю `Gender`, чтобы агрегировать информацию по полу сотрудников.
   - Посчитайте общую сумму зарплат (`Salary`) для каждого пола и назовите этот столбец `sum_salary`.
   - Создайте массив, содержащий уникальные профессии (`Profession`) для каждого пола, используя функцию `groupUniqArray`.
2. **Во внешнем запросе**:
   - Используйте функцию `arrayJoin` для развертывания массива `arr_profession`, чтобы каждая профессия отображалась как отдельная строка. -->

1. **Inside the CTE**:
- Group the data by the `Gender` field to aggregate information by employee gender.
   - Calculate the total amount of salaries (`Salary`) for each gender and name this column `sum_salary'.
   - Create an array containing unique professions (`Profession`) for each gender using the 'groupUniqArray` function.
2. **In an external query**:
- Use the `arrayJoin` function to expand the `arr_profession` array so that each profession is displayed as a separate row.

In [21]:
q = """
with cte as (
    SELECT
    Gender,
    sum(Salary) as sum_salary,
    groupUniqArray(Profession) as arr_profession
    FROM employee_data
    GROUP BY
    Gender
)
SELECT 
  Gender,
  sum_salary,
  arrayJoin(arr_profession) as Profession
FROM cte;
"""

result = client.execute(q)

columns = ['Gender', 'sum_salary', 'Profession']
df = transform_to_dataframe(result, columns)
df.head()


,Gender,sum_salary,Profession
0,F,2568000,Manager
1,F,2568000,Analyst
2,F,2568000,Engineer
3,F,2568000,Senior Specialist
4,F,2568000,Developer


### Views

In [22]:
q = """
CREATE VIEW IF NOT EXISTS employee_summary AS
SELECT Gender, City, COUNT(*) AS total_employees
FROM employee_data
GROUP BY Gender, City;
"""

client.execute(q)

q = """
SELECT *
FROM employee_summary
ORDER BY Gender, City DESC;
"""

result = client.execute(q)

columns = client.execute("DESCRIBE TABLE employee_summary")
columns = [desc[0] for desc in columns]
df = transform_to_dataframe(result, columns)
df.head()

,Gender,City,total_employees
0,F,Yekaterinburg,2
1,F,SPb,2
2,F,Omsk,2
3,F,Novosibirsk,1
4,F,Moscow,1


#### insert into `employee_data` table new data and check the view behavior

In [23]:
q = """
INSERT INTO employee_data (
    Age, 
    Salary, 
    Experience,
    City,
    Gender
)
VALUES
(35, 70000, 10, 'Moscow', 'M'),
(45, NULL, 15, 'SPb', 'F'),
(29, 500000, 7, 'Novosibirsk', 'M'),
(NULL, 80000, 14, 'Yekaterinburg', 'F'),
(40, 850000, 14, 'Kazan', 'M'),
(33, NULL, 11, 'Omsk', 'F'),
(28, 60000, 23, 'Samara', 'M'),
(36, 780000, 13, 'Krasnoyarsk', 'F')
"""

client.execute(q)

q = """
SELECT *
FROM employee_summary
ORDER BY Gender, City DESC;
"""

result = client.execute(q)

columns = client.execute("DESCRIBE TABLE employee_summary")
columns = [desc[0] for desc in columns]
df = transform_to_dataframe(result, columns)
df.head()

,Gender,City,total_employees
0,F,Yekaterinburg,3
1,F,SPb,3
2,F,Omsk,3
3,F,Novosibirsk,1
4,F,Moscow,1


### Materialized views

In [24]:
q = """
CREATE MATERIALIZED VIEW IF NOT EXISTS employee_summary_mv
ENGINE = MergeTree()
ORDER BY (Gender, City) AS
SELECT Gender, City, COUNT(*) AS total_employees
FROM employee_data
GROUP BY Gender, City;
"""

client.execute(q)

# insert into source table (so after that materialized 
# view will be updated and have only new data)
q = """
INSERT INTO employee_data (
    Age, 
    Salary, 
    Experience,
    City
)
VALUES
(35, 70000, 10, 'Moscow'),
(35, 73000, 13, 'Moscow'),
(45, NULL, 15, 'SPb'),
(29, 500000, 7, 'Novosibirsk')
"""

client.execute(q)

q = """
SELECT *
FROM employee_summary_mv

"""


result = client.execute(q)

columns = client.execute("DESCRIBE TABLE employee_summary_mv")
columns = [desc[0] for desc in columns]
df = transform_to_dataframe(result, columns)
df.head()

,Gender,City,total_employees
0,,Moscow,2
1,,Novosibirsk,1
2,,SPb,1


### Drop table

<!-- Удалить всех сотрудников из Москвы -->
#### Delete all employees from Moscow


In [25]:
q = """
DELETE FROM employee_data WHERE City = 'Moscow';
"""

client.execute(q)

q = """
SELECT * FROM employee_data
WHERE City = 'Moscow';
"""

result = client.execute(q)
print(result)

[]


<!-- Очистить таблицу `employee_data`, удалив все записи, но оставив саму таблицу -->
#### Clear the `employee_data` table by deleting all records, but leaving the table itself

In [26]:
q = """
TRUNCATE TABLE employee_data;
"""

client.execute(q)

[]

<!-- Полностью удалить таблицу `employee_data` вместе со всей её структурой и данными -->
#### Completely delete the `employee_data` table along with all its structure and data

In [27]:
q = """
DROP TABLE employee_data;
"""

client.execute(q)

[]

In [28]:
# drop view
q = """
DROP VIEW employee_summary;
"""

client.execute(q)

# drop materialized view
q = """
DROP VIEW employee_summary_mv;
"""

client.execute(q)

[]

### Engine 

<!-- Исследуйте различия в обработке данных между таблицами с движками `MergeTree` и `ReplacingMergeTree`, используя их поведение при наличии дубликатов. Выполните следующие шаги:

1. **Создайте две таблицы** с одинаковой структурой, но разными типами движков:
- Таблицу `employee_data_mt` на базе движка `MergeTree`.
- Таблицу `employee_data_rmt` на базе движка `ReplacingMergeTree`, указав в качестве ключа `Start_Date` для замены дубликатов.
2. **Заполните таблицы данными**:
- Вставьте базовые записи для каждой таблицы, указав значения для `Gender`, `City` и `Start_Date`.
- Добавьте дополнительные записи в `employee_data_rmt`, которые имеют дубликаты по `Start_Date`, чтобы протестировать поведение `ReplacingMergeTree`.
3. **Проанализируйте различия в хранении данных**:
- Выполните запрос, который подсчитает количество записей в каждой таблице
- Проанализируйте результат и ответьте на вопросы:
   - Почему количество записей в `employee_data_mt` и `employee_data_rmt` отличается?
   - Какая таблица хранит уникальные записи по ключу `Start_Date`, а какая сохраняет все вставленные данные, включая дубликаты?
4. **Вывод**:
- Сделайте вывод о том, как каждый из движков обрабатывает дубликаты.
- Определите, в каких случаях будет предпочтительнее использовать `MergeTree` или `ReplacingMergeTree` для таблицы, в зависимости от требований к обновляемости и уникальности данных. -->

Explore the differences in data processing between tables with the `MergeTree` and `ReplacingMergeTree` engines, using their behavior in the presence of duplicates. Follow these steps:

1. **Create two tables** with the same structure, but different types of engines:
- The `employee_data_mt` table based on the 'MergeTree` engine.
- The `employee_data_rmt` table based on the `ReplacingMergeTree` engine, specifying `Start_Date` as the key to replace duplicates.
2. **Fill in the tables with data**:
- Insert the base records for each table, specifying values for `Gender`, `City` and `Start_Date'.
- Add additional entries to `employee_data_rmt` that have duplicates by `Start_Date` to test the behavior of `ReplacingMergeTree'.
3. **Analyze the differences in data storage**:
- Run a query that will count the number of records in each table
- Analyze the result and answer the questions:
   - Why is the number of records in `employee_data_mt` and `employee_data_rmt` different?
- Which table stores unique records by the `Start_Date` key, and which one stores all inserted data, including duplicates?
4. **Conclusion**:
- Make a conclusion about how each of the engines handles duplicates.
- Determine in which cases it would be preferable to use `MergeTree` or `ReplacingMergeTree` for the table, depending on the requirements for updatability and uniqueness of the data.


In [29]:
# Create tables for MergeTree and ReplacingMergeTree
client.execute("""
 CREATE TABLE IF NOT EXISTS employee_data_mt (
   Gender String,
   City String,
   Start_Date DateTime
 )
 ENGINE = MergeTree()
 ORDER BY (Gender, City);
""")

client.execute("""
 CREATE TABLE IF NOT EXISTS employee_data_rmt (
   Gender String,
   City String,
   Start_Date DateTime
 )
 ENGINE = ReplacingMergeTree(Start_Date)
 ORDER BY (Gender, City);
""")

# Insert data into tables
client.execute("""
INSERT INTO employee_data_mt (Gender, City, Start_Date) VALUES 
    ('M', 'Moscow', '2023-01-01 08:00:00'),
    ('F', 'SPb', '2023-01-01 09:00:00'),
    ('M', 'Kazan', '2023-01-01 10:00:00'),
    ('F', 'Moscow', '2023-01-01 11:00:00'),
    ('M', 'SPb', '2023-01-01 12:00:00');
""")

client.execute("""
INSERT INTO employee_data_rmt (Gender, City, Start_Date) VALUES 
    ('M', 'Moscow', '2023-01-01 08:00:00'),
    ('F', 'SPb', '2023-01-01 09:00:00'),
    ('M', 'Kazan', '2023-01-01 10:00:00'),
    ('F', 'Moscow', '2023-01-01 11:00:00'),
    ('M', 'SPb', '2023-01-01 12:00:00');
""")

# Add duplicates to the employee_data_mt table for testing MergeTree
client.execute("""
INSERT INTO employee_data_mt (Gender, City, Start_Date) VALUES 
    ('F', 'Moscow', '2023-01-01 11:00:00'),  
    ('M', 'SPb', '2023-01-01 12:00:00'),   
    ('M', 'Moscow', '2023-01-01 08:00:00');
""")

# Add duplicates to the employee_data_rmt table for testing ReplacingMergeTree
client.execute("""
INSERT INTO employee_data_rmt (Gender, City, Start_Date) VALUES 
    ('F', 'Moscow', '2023-01-01 11:00:00'),  
    ('M', 'SPb', '2023-01-01 12:00:00'),   
    ('M', 'Moscow', '2023-01-01 08:00:00'); 


OPTIMIZE TABLE employee_data_rmt FINAL;

""")

# Check the number of rows in the tables

client.execute("""
SELECT 
  (SELECT COUNT() FROM employee_data_mt) as cnt_mt,
  (SELECT COUNT() FROM employee_data_rmt) as cnt_rmt;
""")

[(8, 8)]

In [30]:
# drop tables

client.execute("""
DROP TABLE employee_data_mt;
""")
client.execute("""
DROP TABLE employee_data_rmt;
""")


[]